<a href="https://colab.research.google.com/github/A-b-h-a-y-0-2/Computer-Vision/blob/main/Fine-tuning%20ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install lightning==2.2.1


  Using cached lightning-2.2.1-py3-none-any.whl (2.1 MB)
  Using cached lightning_utilities-0.11.3.post0-py3-none-any.whl (26 kB)
  Using cached torchmetrics-1.4.0.post0-py3-none-any.whl (868 kB)
  Using cached pytorch_lightning-2.3.3-py3-none-any.whl (812 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.10

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchmetrics
import lightning as L

In [5]:
!pip install -q kaggle


In [6]:
!mkdir ~/.kaggle


In [7]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json


cp: cannot stat '/content/kaggle.json': No such file or directory


In [8]:
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d apollo2506/eurosat-dataset

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/apollo2506/eurosat-dataset
License(s): CC0-1.0
 99% 2.02G/2.04G [00:28<00:00, 97.7MB/s]
100% 2.04G/2.04G [00:28<00:00, 77.3MB/s]


In [26]:
!unzip /content/eurosat-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: EuroSATallBands/River/River_1990.tif  
  inflating: EuroSATallBands/River/River_1991.tif  
  inflating: EuroSATallBands/River/River_1992.tif  
  inflating: EuroSATallBands/River/River_1993.tif  
  inflating: EuroSATallBands/River/River_1994.tif  
  inflating: EuroSATallBands/River/River_1995.tif  
  inflating: EuroSATallBands/River/River_1996.tif  
  inflating: EuroSATallBands/River/River_1997.tif  
  inflating: EuroSATallBands/River/River_1998.tif  
  inflating: EuroSATallBands/River/River_1999.tif  
  inflating: EuroSATallBands/River/River_2.tif  
  inflating: EuroSATallBands/River/River_20.tif  
  inflating: EuroSATallBands/River/River_200.tif  
  inflating: EuroSATallBands/River/River_2000.tif  
  inflating: EuroSATallBands/River/River_2001.tif  
  inflating: EuroSATallBands/River/River_2002.tif  
  inflating: EuroSATallBands/River/River_2003.tif  
  inflating: EuroSATallBands/River/River_2004.tif  
  inflating: EuroSA

In [4]:
import transformers
print(transformers.__version__)


4.41.2


In [53]:
from transformers import ViTForImageClassification,ViTFeatureExtractor,ViTImageProcessor


In [78]:
class VisionTransformerPretrained(L.LightningModule):
    def __init__(self, model = 'google/vit-base-patch16-224',num_classes=1000,learning_rate=1e-4):
        super().__init__()
        self.feature_extractor = ViTImageProcessor.from_pretrained(model)
        backbone = ViTForImageClassification().from_pretrained(model,num_classes = 10, ignore_mismatched_sizes= True,config = default_config)
        self.backbone = backbone
        self.acc = torchmetrics.Accuracy('multiclass', num_classes = num_classes)
        self.f1 = torchmetrics.F1Score('multiclass', num_classes = num_classes)
        self.precision = torchmetrics.Precision('multiclass', num_classes = num_classes)
        self.recall = torchmetrics.Recall('multiclass', num_classes = num_classes)
        self.learning_rate = learning_rate
    def forward(self,x):
      return self.backbone(x)

    def step(self ,batch):
      x,y = batch
      prediction = self.backbone(x)
      y_hat = torch.argmax(prediction.logits,dim = 1)
      loss = F.cross_entropy(prediction.logits,y)
      acc = self.acc(prediction.logits,y)
      f1 = self.f1(prediction.logits,y)
      precision = self.precision(prediction.logits,y)
      recall = self.recall(prediction.logits,y)
      return loss,acc,f1,precision,recall,y_hat,y

    def training_step(self,batch,batch_idx):
      loss,acc,f1,precision,recall,y_hat,y = self.step(batch)
      self.log("train_loss",loss)
      return loss

    def validation_step(self,batch,batch_idx):
      loss,acc,f1,precision,recall,y_hat,y = self.step(batch)
      self.log("val_acc",acc,on_epoch=True,on_step = False)

    def configure_optimizers(self):
      optimizer = optim.Adam(self.parameters(),lr = 1e-4)
      return optimizer

In [79]:
from torch.utils.data import DataLoader,Subset
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder

class EuroSAT_RGB_DataModule(L.LightningDataModule):
  def __init__(self,data_dir,batch_size):
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
  def setup(self,stage = None):
    transforms = v2.Compose([
        v2.Resize((224,224),interpolation = 2),
        v2.ToDtype(torch.float32,scale = True),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])
    data = ImageFolder(self.data_dir,transform = transforms)
    targets = np.asarray(data.targets)

    train_ix,test_ix = train_test_split(np.arange(len(data.targets)),test_size= 5400,stratify = targets)
    train_ix,val_ix = train_test_split(train_ix,test_size = 2700,stratify = targets[train_ix])

    self.train_dataset = Subset(data,train_ix)
    self.val_dataset = Subset(data,val_ix)
    self.test_dataset = Subset(data,test_ix)
  @property
  def num_classes(self):
    return 10

  def train_dataloader(self) :
    return DataLoader(self.train_dataset,batch_size = self.batch_size,shuffle = True)

  def val_dataloader(self):
    return DataLoader(self.val_dataset,batch_size = self.batch_size,shuffle = False)

  def test_dataloader(self):
    return DataLoader(self.test_dataset,batch_size = self.batch_size,shuffle = False)


In [80]:
import yaml
with open('/content/eurosat.yaml') as cf_file:
  default_config = yaml.safe_load(cf_file.read())

In [85]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

if __name__ == '__main__':
  L.seed_everything(121)
  dm = EuroSAT_RGB_DataModule(data_dir = '/content/EuroSAT',batch_size = 32)
  dm.prepare_data()
  dm.setup()

  train_dataloader = dm.train_dataloader()
  val_dataloader = dm.val_dataloader()
  test_dataloader = dm.test_dataloader()

  model = VisionTransformerPretrained('google/vit-base-patch16-224',dm.num_classes, default_config.get('learning_rate', 1e-4))
  early_stopping = EarlyStopping(monitor = 'valid_acc', patience=6 , mode = 'max')
  logger = TensorBoardLogger(save_dir = '/content/logs',name = 'eurosat')
  trainer = L.trainer(devices=1,callbacks = [early_stopping],logger = logger)
  trainer.fit(model=model,train_dataloaders = train_dataloader,val_dataloader = val_dataloader)
  trainer.test(model=model,dataloaders = test_dataloader,verbose = True)


INFO: Seed set to 121
INFO:lightning.fabric.utilities.seed:Seed set to 121


TypeError: ViTForImageClassification.__init__() missing 1 required positional argument: 'config'

In [59]:
import lightning.pytorch as L
import torch
import torch.nn.functional as F
import torchmetrics
import numpy as np
from transformers import ViTFeatureExtractor, ViTForImageClassification, ViTImageProcessor
from torch import optim

class VisionTransformerPretrained(L.LightningModule):
    def __init__(self, model='google/vit-base-patch16-224', num_classes=1000, learning_rate=1e-4):
        super().__init__()
        self.feature_extractor = ViTImageProcessor.from_pretrained(model)
        self.backbone = ViTForImageClassification.from_pretrained(model, num_labels=num_classes, ignore_mismatched_sizes=True)
        self.acc = torchmetrics.Accuracy('multiclass', num_classes=num_classes)
        self.f1 = torchmetrics.F1Score('multiclass', num_classes=num_classes)
        self.precision = torchmetrics.Precision('multiclass', num_classes=num_classes)
        self.recall = torchmetrics.Recall('multiclass', num_classes=num_classes)
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.backbone(x)

    def step(self, batch):
        x, y = batch
        prediction = self.backbone(x)
        y_hat = torch.argmax(prediction.logits, dim=1)
        loss = F.cross_entropy(prediction.logits, y)
        acc = self.acc(prediction.logits, y)
        f1 = self.f1(prediction.logits, y)
        precision = self.precision(prediction.logits, y)
        recall = self.recall(prediction.logits, y)
        return loss, acc, f1, precision, recall, y_hat, y

    def training_step(self, batch, batch_idx):
        loss, acc, f1, precision, recall, y_hat, y = self.step(batch)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc, f1, precision, recall, y_hat, y = self.step(batch)
        self.log("val_acc", acc, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

from torch.utils.data import DataLoader, Subset
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder

class EuroSAT_RGB_DataModule(L.LightningDataModule):
    def __init__(self, data_dir, batch_size):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage=None):
        transforms = v2.Compose([
            v2.Resize((224, 224), interpolation=2),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        data = ImageFolder(self.data_dir, transform=transforms)
        targets = np.asarray(data.targets)

        train_ix, test_ix = train_test_split(np.arange(len(data.targets)), test_size=5400, stratify=targets)
        train_ix, val_ix = train_test_split(train_ix, test_size=2700, stratify=targets[train_ix])

        self.train_dataset = Subset(data, train_ix)
        self.val_dataset = Subset(data, val_ix)
        self.test_dataset = Subset(data, test_ix)

    @property
    def num_classes(self):
        return 10

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

if __name__ == '__main__':
    L.seed_everything(121)
    dm = EuroSAT_RGB_DataModule(data_dir='/content/EuroSAT', batch_size=32)
    dm.prepare_data()
    dm.setup()

    train_dataloader = dm.train_dataloader()
    val_dataloader = dm.val_dataloader()
    test_dataloader = dm.test_dataloader()

    model = VisionTransformerPretrained('google/vit-base-patch16-224', dm.num_classes, learning_rate=1e-4)
    early_stopping = EarlyStopping(monitor='val_acc', patience=6, mode='max')
    logger = TensorBoardLogger(save_dir='/content/logs', name='eurosat')
    trainer = L.Trainer(devices=1, callbacks=[early_stopping], logger=logger)
    trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
    trainer.test(model=model, dataloaders=test_dataloader, verbose=True)


INFO: Seed set to 121
INFO:lightning.fabric.utilities.seed:Seed set to 121


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

TypeError: Normalize() does not support PIL images.

In [94]:
from transformers import ViTForImageClassification, ViTImageProcessor, ViTConfig
import torchmetrics
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as L
import numpy as np
import yaml
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split

class VisionTransformerPretrained(L.LightningModule):
    def __init__(self, model='google/vit-base-patch16-224', num_classes=1000, learning_rate=1e-4):
        super().__init__()
        self.feature_extractor = ViTImageProcessor.from_pretrained(model)
        config = ViTConfig.from_pretrained(model)
        config.num_labels = num_classes
        backbone = ViTForImageClassification.from_pretrained(model, config=config, ignore_mismatched_sizes=True)
        self.backbone = backbone
        self.acc = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        self.f1 = torchmetrics.F1Score(task='multiclass', num_classes=num_classes)
        self.precision = torchmetrics.Precision(task='multiclass', num_classes=num_classes)
        self.recall = torchmetrics.Recall(task='multiclass', num_classes=num_classes)
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.backbone(x)

    def step(self, batch):
        x, y = batch
        prediction = self.backbone(x)
        y_hat = torch.argmax(prediction.logits, dim=1)
        loss = F.cross_entropy(prediction.logits, y)
        acc = self.acc(prediction.logits, y)
        f1 = self.f1(prediction.logits, y)
        precision = self.precision(prediction.logits, y)
        recall = self.recall(prediction.logits, y)
        return loss, acc, f1, precision, recall, y_hat, y

    def training_step(self, batch, batch_idx):
        loss, acc, f1, precision, recall, y_hat, y = self.step(batch)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc, f1, precision, recall, y_hat, y = self.step(batch)
        self.log("val_acc", acc, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

class EuroSAT_RGB_DataModule(L.LightningDataModule):
    def __init__(self, data_dir, batch_size):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage=None):
        transforms = v2.Compose([
            v2.Resize((224, 224), interpolation=2),
            v2.ToTensor(),
            v2.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        data = ImageFolder(self.data_dir, transform=transforms)
        targets = np.asarray(data.targets)

        train_ix, test_ix = train_test_split(np.arange(len(data.targets)), test_size=5400, stratify=targets)
        train_ix, val_ix = train_test_split(train_ix, test_size=2700, stratify=targets[train_ix])

        self.train_dataset = Subset(data, train_ix)
        self.val_dataset = Subset(data, val_ix)
        self.test_dataset = Subset(data, test_ix)

    @property
    def num_classes(self):
        return 10

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

with open('/content/eurosat.yaml') as cf_file:
    default_config = yaml.safe_load(cf_file.read())

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

if __name__ == '__main__':
    L.seed_everything(121)
    dm = EuroSAT_RGB_DataModule(data_dir='/content/EuroSAT', batch_size=32)
    dm.prepare_data()
    dm.setup()

    train_dataloader = dm.train_dataloader()
    val_dataloader = dm.val_dataloader()
    test_dataloader = dm.test_dataloader()

    model = VisionTransformerPretrained('google/vit-base-patch16-224', dm.num_classes, default_config.get('learning_rate', 1e-4))
    early_stopping = EarlyStopping(monitor='val_acc', patience=6, mode='max')
    logger = TensorBoardLogger(save_dir='/content/logs', name='eurosat')
    trainer = L.Trainer(devices=1, callbacks=[early_stopping], logger=logger)
    trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
    trainer.test(model=model, dataloaders=test_dataloader, verbose=True)



INFO:lightning_fabric.utilities.seed:Seed set to 121
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


MisconfigurationException: No `test_step()` method defined to run `Trainer.test`.